In [1]:
import pandas as pd
from pathlib import Path
from typing import List
from sklearn.pipeline import Pipeline
from feature_engine.imputation import CategoricalImputer, MeanMedianImputer
from sklearn.base import BaseEstimator, TransformerMixin

In [2]:
PACKAGE_ROOT = Path().resolve().parents[0]
data = pd.read_csv(f'{PACKAGE_ROOT}/datasets/house_price.csv', index_col=0)
data.head()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,,
7229300521,20141013T000000,231300.0,2,1.00,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


### Transformer

In [3]:
class DataTypeTransformer(BaseEstimator, TransformerMixin):
    """Data type transformer."""
    def __init__(self, variables: List[str], data_type: str):

        if not isinstance(variables, list):
            raise ValueError("variables should be a list")

        self.variables = variables
        self.data_type = data_type
    
    
    def fit(self, X: pd.DataFrame, y: pd.Series = None):
        # we need this step to fit the sklearn pipeline
        return self
    
    def transform(self, X: pd.DataFrame) -> pd.DataFrame:

        # so that we do not over-write the original dataframe
        X = X.copy()

        for var in self.variables:

            if self.data_type == "categorical":
                X[var] = X[var].astype(str)
            elif self.data_type == "numerical":
                X[var] = X[var].astype(int)
            elif self.data_type == "date":
                X[var] = pd.to_datetime(X[var])
            
        return X

In [4]:
demo_pipe = Pipeline(
    [
        (
            "categorical",
            DataTypeTransformer(
                variables=['zipcode', 'waterfront', 'view', 'condition', 'grade'],
                data_type="categorical"
            )
        ),
        (
            "numerical",
            DataTypeTransformer(
                variables=['bedrooms','bathrooms','sqft_living','sqft_lot', 'floors',
                           'sqft_above','sqft_basement','sqft_living15'],
                data_type="numerical"
            )
        ),
        (
            "categorical_missing",
            CategoricalImputer(
                imputation_method="missing",
                #variables=config.data_config.categorical_variables
                variables=['zipcode', 'waterfront', 'view', 'condition', 'grade']
            )
        ),
        (
            "numerical_missing",
            MeanMedianImputer(
                imputation_method="mean",
                #variables=config.data_config.numerical_variables
                variables=['bedrooms','bathrooms','sqft_living','sqft_lot','floors',
                           'sqft_above', 'sqft_basement','sqft_living15']
            )
        )
    ]
)

In [5]:
demo_pipe.fit_transform(data)

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,,
7229300521,20141013T000000,231300.0,2,1,1180,5650,1,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
6414100192,20141209T000000,538000.0,3,2,2570,7242,2,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
5631500400,20150225T000000,180000.0,2,1,770,10000,1,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2487200875,20141209T000000,604000.0,4,3,1960,5000,1,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
1954400510,20150218T000000,510000.0,3,2,1680,8080,1,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263000018,20140521T000000,360000.0,3,2,1530,1131,3,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
6600060120,20150223T000000,400000.0,4,2,2310,5813,2,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
1523300141,20140623T000000,402101.0,2,0,1020,1350,2,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007


In [6]:
import pickle #Alternativa DILL
filename = '../assets/pipeline.pkl'
pickle.dump(demo_pipe, open(filename, 'wb'))

### Reaproveitando o pipeline com pickle

In [7]:
pipe = pickle.load(open(filename, 'rb'))

In [8]:
pipe.fit_transform(data)

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,,,,,,,,,,,,,,,,,,,,
7229300521,20141013T000000,231300.0,2,1,1180,5650,1,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
6414100192,20141209T000000,538000.0,3,2,2570,7242,2,0,0,3,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
5631500400,20150225T000000,180000.0,2,1,770,10000,1,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
2487200875,20141209T000000,604000.0,4,3,1960,5000,1,0,0,5,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
1954400510,20150218T000000,510000.0,3,2,1680,8080,1,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263000018,20140521T000000,360000.0,3,2,1530,1131,3,0,0,3,8,1530,0,2009,0,98103,47.6993,-122.346,1530,1509
6600060120,20150223T000000,400000.0,4,2,2310,5813,2,0,0,3,8,2310,0,2014,0,98146,47.5107,-122.362,1830,7200
1523300141,20140623T000000,402101.0,2,0,1020,1350,2,0,0,3,7,1020,0,2009,0,98144,47.5944,-122.299,1020,2007
